In [56]:
# Khai báo các thư viện cần thiết

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import pickle
import numpy as np
import torch.optim as optim
import os

In [57]:
# Khai báo tên model
while True:
    model_name = input("Enter model name: ")
    if model_name == "":
        print("Model name cannot be empty. Please enter a valid name.")
    else:
        if not os.path.exists(model_name):
            print("Model is not exist. Please enter a valid name.")
        else:
            break

In [58]:
# Mạng Nơ ron 

class MazeNetCombined(nn.Module):
    def __init__(self, local_size=11, global_size=10, num_actions=4):
        super(MazeNetCombined, self).__init__()
        
        # Quan sát cục bộ
        self.conv1_local = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2_local = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3_local = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4_local = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        
        # Quan sát toàn cục
        self.conv1_global = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2_global = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3_global = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4_global = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        
        # Fully Connected cho vị trí hiện tại
        self.fc_position = nn.Linear(2, 32)

        # Tầng Fully Connected cuối cùng
        self.fc1 = nn.Linear(256 * local_size * local_size + 256 * global_size * global_size + 32, 256)
        self.fc2 = nn.Linear(256, 128)
        self.dropout_fc = nn.Dropout(p=0.5)  # Dropout trước tầng FC3
        self.fc3 = nn.Linear(128, num_actions)

    def forward(self, local_obs, global_obs, position):
        # Xử lý local_obs
        x_local = F.relu(self.conv1_local(local_obs))
        x_local = F.relu(self.conv2_local(x_local))
        x_local = F.relu(self.conv3_local(x_local))
        x_local = F.relu(self.conv4_local(x_local))
        x_local = x_local.view(x_local.size(0), -1)
    
        # Xử lý global_obs
        x_global = F.relu(self.conv1_global(global_obs))
        x_global = F.relu(self.conv2_global(x_global))
        x_global = F.relu(self.conv3_global(x_global))
        x_global = F.relu(self.conv4_global(x_global))
        x_global = x_global.view(x_global.size(0), -1)
        # Xử lý vị trí hiện tại
        x_position = F.relu(self.fc_position(position))

        # Kết hợp tất cả
        x = torch.cat((x_local, x_global, x_position), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.dropout_fc(x)  # Dropout trước FC3
        x = self.fc3(x)

        return x


In [59]:
# Replay Buffer
class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []

    def push(self, experience):
        """Thêm một trải nghiệm (local_obs, global_obs, position, action, reward, next_local_obs, next_global_obs, next_position, done) vào replay buffer."""
        if len(self.buffer) >= self.capacity:
            self.buffer.pop(0)  # Loại bỏ trải nghiệm cũ nhất nếu đầy
        self.buffer.append(experience)

    def sample(self, batch_size):
        """Trích xuất batch mẫu từ replay buffer."""
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        batch = [self.buffer[idx] for idx in indices]
        # Tách dữ liệu thành các phần riêng biệt
        local_obs, global_obs, position, actions, rewards, next_local_obs, next_global_obs, next_position, dones = zip(*batch)
        return (np.array(local_obs), np.array(global_obs), np.array(position), 
                np.array(actions), np.array(rewards), np.array(next_local_obs), 
                np.array(next_global_obs), np.array(next_position), np.array(dones))

    def __len__(self):
        return len(self.buffer)
    
    def extend(self, buffer):
        """Mở rộng replay buffer bằng cách thêm một buffer khác."""
        if len(self.buffer) + len(buffer.buffer) > self.capacity:
            self.capacity = len(self.buffer) + len(buffer.buffer)
        self.buffer.extend(buffer.buffer)

In [60]:
# Cập nhật model
def update_model(policy_net, target_net, replay_buffer, optimizer, batch_size, gamma, device):
    """
    Cập nhật mô hình chính (policy network) cho mạng có đầu vào đa dạng (local_obs, global_obs, position).

    Args:
    - policy_net (nn.Module): Mạng chính dự đoán giá trị Q(s, a).
    - target_net (nn.Module): Mạng mục tiêu dùng để tính Q_target.
    - replay_buffer (ReplayBuffer): Bộ nhớ replay chứa các kinh nghiệm (local_obs, global_obs, position, action, reward, next_local_obs, next_global_obs, next_position, done).
    - optimizer (torch.optim.Optimizer): Trình tối ưu hóa (Adam, SGD, ...).
    - batch_size (int): Kích thước batch mẫu từ replay buffer.
    - gamma (float): Hệ số chiết khấu (discount factor).
    - device (torch.device): Thiết bị thực thi (CPU hoặc GPU).

    Returns:
    - loss (float): Giá trị mất mát (loss) sau khi cập nhật.
    """
    # Kiểm tra nếu replay buffer chưa đủ dữ liệu
    if len(replay_buffer) < batch_size:
        return None

    # 1. Lấy mẫu từ replay buffer
    batch = replay_buffer.sample(batch_size)
    (local_obs, global_obs, position, actions, rewards, 
     next_local_obs, next_global_obs, next_position, dones) = batch

    # 2. Chuyển đổi dữ liệu sang Tensor và đưa vào thiết bị (CPU/GPU)
    local_obs = torch.tensor(local_obs, dtype=torch.float32).to(device).unsqueeze(1)  # Thêm chiều kênh cho CNN
    global_obs = torch.tensor(global_obs, dtype=torch.float32).to(device).unsqueeze(1)  # Thêm chiều kênh
    position = torch.tensor(position, dtype=torch.float32).to(device)
    actions = torch.tensor(actions, dtype=torch.long).to(device)
    rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
    next_local_obs = torch.tensor(next_local_obs, dtype=torch.float32).to(device).unsqueeze(1)  # Thêm chiều kênh
    next_global_obs = torch.tensor(next_global_obs, dtype=torch.float32).to(device).unsqueeze(1)  # Thêm chiều kênh
    next_position = torch.tensor(next_position, dtype=torch.float32).to(device)
    dones = torch.tensor(dones, dtype=torch.float32).to(device)

    # 3. Dự đoán Q(s, a) từ policy_net
    q_values = policy_net(local_obs, global_obs, position)  # Đầu ra: (batch_size, num_actions)
    q_values = q_values.gather(1, actions.unsqueeze(1)).squeeze(1)  # Lấy giá trị Q(s, a) cho hành động đã thực hiện

    # 4. Tính toán Q_target bằng target_net
    with torch.no_grad():
        next_q_values = target_net(next_local_obs, next_global_obs, next_position)  # Dự đoán Q(s', a') từ target_net
        max_next_q_values = next_q_values.max(1)[0]  # Lấy giá trị lớn nhất Q(s', a')
        q_targets = rewards + gamma * max_next_q_values * (1 - dones)  # Hàm Bellman

    # 5. Tính hàm mất mát
    loss = F.mse_loss(q_values, q_targets)

    # 6. Tối ưu hóa mô hình
    optimizer.zero_grad()  # Xóa gradient cũ
    loss.backward()  # Lan truyền ngược (backpropagation)
    optimizer.step()  # Cập nhật trọng số

    return loss.item()


In [61]:
# Đồng bộ với mạng mục tiêu
def sync_target_network(q_network, target_network):
    target_network.load_state_dict(q_network.state_dict())

In [62]:
# Phương thức lưu, tải trọng số của mô hình

# Lưu trọng số của mô hình
def save_model(model, path):
    torch.save(model, path)
    print(f"Model saved to: {path}")
    
# Tải trọng số của mô hình
def load_model(path, device = "cuda"):
    policy_model = torch.load(path, map_location=device, weights_only=False)  # Tải mô hình từ file
    target_model = policy_model
    target_model.eval() # Đặt target_model ở chế độ đánh giá
    print(F"Model loaded from: {path}")
    return policy_model, target_model

In [63]:
# Khởi tạo các siêu tham số

# Thiết bị thực thi (CPU hoặc GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Training on: {device}")

# Đọc training info từ file
with open(model_name + "/training_info.pkl", "rb") as f:
    training_info = pickle.load(f)
gamma = training_info["gamma"]  # Hệ số chiết khấu
learning_rate = training_info["learning_rate"]  # Tốc độ học
weight_decay = training_info["weight_decay"]  # Hệ số điều chỉnh trọng số

# Hyperparameters
batch_size = 256  # Kích thước batch khi lấy mẫu từ replay buffer
max_episode = 10000             # Số episode tối đa

# Tải model từ file 
policy_net, target_net = load_model(model_name + "/model.pth", device=device)

# Optimizer
optimizer = optim.Adam(policy_net.parameters(), lr=learning_rate, weight_decay = weight_decay)

print("Initialization complete!")

Training on: cuda
Model loaded from: model14/model.pth
Initialization complete!


In [64]:
# Khởi tạo replay buffer
replay_buffer = ReplayBuffer(capacity=0)
with open(model_name + "/replay_buffer.pkl", "rb") as file:
    while True:
        try:
            # Tải buffer từ tệp pickle
            buffer = pickle.load(file)
            # Thêm buffer vào replay buffer hiện tại
            replay_buffer.extend(buffer)
        except EOFError:
            # Đến cuối tệp
            break
print(f"Replay buffer loaded with {len(replay_buffer)} experiences.")

Replay buffer loaded with 17914 experiences.


In [65]:
# Huấn luyện mô hình trên replay buffer
for episode in range(max_episode):
    print(f"Episode {episode + 1}/{max_episode}")
    loss = update_model(policy_net, target_net, replay_buffer, optimizer, batch_size, gamma, device)
    
    if loss is not None:
        print(f"Loss: {loss:.4f}")
    else:
        print("Replay buffer not enough data for training.")

    # Đồng bộ hóa mạng mục tiêu
    if episode % 100 == 0:
        sync_target_network(policy_net, target_net)

Episode 1/10000
Loss: 88.6658
Episode 2/10000
Loss: 135.1713
Episode 3/10000
Loss: 98.6227
Episode 4/10000
Loss: 69.4593
Episode 5/10000
Loss: 101.1917
Episode 6/10000
Loss: 61.1726
Episode 7/10000
Loss: 88.3190
Episode 8/10000
Loss: 134.5222
Episode 9/10000
Loss: 116.6969
Episode 10/10000
Loss: 99.3611
Episode 11/10000
Loss: 87.0511
Episode 12/10000
Loss: 91.6546
Episode 13/10000
Loss: 63.4470
Episode 14/10000
Loss: 45.9682
Episode 15/10000
Loss: 103.7058
Episode 16/10000
Loss: 56.7056
Episode 17/10000
Loss: 75.7400
Episode 18/10000
Loss: 108.6419
Episode 19/10000
Loss: 67.2243
Episode 20/10000
Loss: 74.7909
Episode 21/10000
Loss: 57.8380
Episode 22/10000
Loss: 58.5514
Episode 23/10000
Loss: 102.2936
Episode 24/10000
Loss: 65.2575
Episode 25/10000
Loss: 66.3813
Episode 26/10000
Loss: 77.5759
Episode 27/10000
Loss: 120.3822
Episode 28/10000
Loss: 67.4614
Episode 29/10000
Loss: 87.1963
Episode 30/10000
Loss: 68.3342
Episode 31/10000
Loss: 64.9031
Episode 32/10000
Loss: 63.0358
Episode 3

In [66]:
# Lưu mô hình
save_model(policy_net, path=model_name + "/model.pth")

Model saved to: model14/model.pth
